## 导入必要的库

In [ ]:
import torch
import torch.library

## 定义自定义算子 Schema

这里的 `'my_ops::custom_add'` 就是我们在 OperatorHandle 中查找的名字

In [ ]:
torch.library.define("my_ops::custom_add", "(Tensor x, Tensor y) -> Tensor")

## 注册 CPU 后端的实现

对应 M x N 中的 N=CPU

In [ ]:
@torch.library.impl("my_ops::custom_add", "CPU")
def custom_add_cpu(x, y):
    print("[Backend: CPU] 正在执行底层的加法计算...")
    return x + y

## 注册 Autograd 后端的实现

对应 M x N 中的 N=Autograd

Autograd 层负责处理反向传播逻辑，然后 "Redispatch" 给底层

In [ ]:
@torch.library.impl("my_ops::custom_add", "Autograd")
def custom_add_autograd(ctx, x, y):
    print("[Backend: Autograd] 拦截到了调用！正在记录计算图...")
    # 在这里，PyTorch 内部会做一个 exclude(Autograd) 的操作，
    # 然后再次调用算子，这次因为没有了 Autograd Key，就会掉落到上面的 CPU 实现中。
    # 为了模拟这个过程，我们直接调用 torch.ops.my_ops.custom_add
    # (注意：真实的 C++ 实现中通过 ctx.save_for_backward 等机制处理，这里是逻辑演示)

    with torch._C._DisableAutograd():  # 模拟 Redispatch 时的 Key 剔除
        result = torch.ops.my_ops.custom_add(x, y)

    print("[Backend: Autograd] 底层计算完毕，返回结果。")
    return result

## 测试代码

### 场景 1: 普通 CPU Tensor（不求导）

分发逻辑: Key(CPU) -> 匹配到 custom_add_cpu

In [ ]:
print("--- 场景 1: 普通 CPU Tensor (不求导) ---")
t1 = torch.ones(2)
t2 = torch.ones(2)
res1 = torch.ops.my_ops.custom_add(t1, t2)
print(f"Result: {res1}\n")

### 场景 2: 需要求导的 Tensor

分发逻辑: 
- Key(Autograd, CPU) 
- -> 优先匹配 Autograd 
- -> custom_add_autograd
- -> (Redispatch) 
- -> Key(CPU) 
- -> custom_add_cpu

In [ ]:
print("--- 场景 2: 需要求导的 Tensor (Requires Grad) ---")
t3 = torch.ones(2, requires_grad=True)
t4 = torch.ones(2)
res2 = torch.ops.my_ops.custom_add(t3, t4)
print(f"Result: {res2}")